<a href="https://colab.research.google.com/github/manjunathreddy25/Final_Year_Project/blob/main/Trader_Behavior_%26_Market_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Project: Trader Behavior & Market Sentiment

In [4]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# --- 1. DATA LOADING ---
FGI_PATH = 'fear_greed_index.csv'
TRADER_DATA_PATH = 'historical_data.csv'

df_fgi = pd.read_csv(FGI_PATH)
df_trader = pd.read_csv(TRADER_DATA_PATH)
print("✅ Data Loading Successful. Starting preprocessing...")

# --- 2. DATA CLEANING AND DAILY AGGREGATION ---

# Convert 'Timestamp IST' to datetime, handling the specified format
df_trader['Timestamp IST'] = pd.to_datetime(df_trader['Timestamp IST'], format='%d-%m-%Y %H:%M')
df_trader['Date'] = df_trader['Timestamp IST'].dt.normalize()

# Convert 'Closed PnL' to numeric and rename
df_trader.rename(columns={'Closed PnL': 'closedPnL'}, inplace=True)
df_trader['closedPnL'] = pd.to_numeric(df_trader['closedPnL'], errors='coerce')
df_trader.dropna(subset=['closedPnL'], inplace=True)

# CLEANING SENTIMENT DATA (FGI)
df_fgi.rename(columns={'classification': 'Sentiment_Classification'}, inplace=True)
df_fgi['Date'] = pd.to_datetime(df_fgi['date']).dt.normalize()
df_fgi.drop_duplicates(subset=['Date'], keep='first', inplace=True)


# --- 3. AGGREGATING TRADER DATA TO DAILY LEVEL ---
agg_funcs = {
    'closedPnL': ['sum', 'count'],
    'Account': 'nunique',
}
daily_performance = df_trader.groupby('Date').agg(agg_funcs)

# Flatten MultiIndex columns and rename
daily_performance.columns = ['_'.join(col).strip() for col in daily_performance.columns.values]
daily_performance.rename(columns={
    'closedPnL_sum': 'Total_Daily_PnL',
    'closedPnL_count': 'Total_Trades',
    'Account_nunique': 'Active_Traders'
}, inplace=True)
daily_performance = daily_performance.reset_index()

# ADDING TRADE DIRECTION BIAS
daily_side_counts = df_trader.groupby(['Date', 'Side']).size().unstack(fill_value=0)
daily_performance = daily_performance.merge(daily_side_counts, on='Date', how='left').fillna(0)

# Calculate Net Buy Bias (handling case-sensitivity from original data)
buy_col = [col for col in daily_performance.columns if 'Buy' in col or 'BUY' in col][0]
sell_col = [col for col in daily_performance.columns if 'Sell' in col or 'SELL' in col][0]
daily_performance['Net_Buy_Bias'] = daily_performance[buy_col] - daily_performance[sell_col]

# --- 4. MERGE DATASETS ---
merged_df = daily_performance.merge(df_fgi[['Date', 'Sentiment_Classification']], on='Date', how='inner')
print("✅ Data Merged. Starting final analysis.")

# --- 5. GROUP AND ANALYZE PERFORMANCE BY SENTIMENT ---
sentiment_analysis = merged_df.groupby('Sentiment_Classification').agg(
    # PnL Metrics
    Avg_Total_Daily_PnL=('Total_Daily_PnL', 'mean'),

    # Activity Metrics
    Avg_Daily_Trades=('Total_Trades', 'mean'),
    Avg_Net_Buy_Bias=('Net_Buy_Bias', 'mean'),
    Avg_Active_Traders=('Active_Traders', 'mean'),

    # Consistency
    Winning_Days=('Total_Daily_PnL', lambda x: (x > 0).sum()),
    Total_Days=('Total_Daily_PnL', 'count')
)

# Calculate Daily Win Rate
sentiment_analysis['Daily_Win_Rate_%'] = (sentiment_analysis['Winning_Days'] / sentiment_analysis['Total_Days']) * 100

# Sort and format the final table for the report
final_report = sentiment_analysis.sort_values(by='Avg_Total_Daily_PnL', ascending=False)
final_report = final_report[[
    'Total_Days',
    'Avg_Total_Daily_PnL',
    'Daily_Win_Rate_%',
    'Avg_Daily_Trades',
    'Avg_Net_Buy_Bias',
    'Avg_Active_Traders'
]]

final_report.columns = [
    'Days Observed',
    'Avg. Total Daily PnL (USD)',
    'Daily Win Rate (%)',
    'Avg. Daily Trades',
    'Avg. Net Buy Trades',
    'Avg. Active Traders'
]

print("\n--- Final Sentiment Performance Report ---")
print(final_report)

/tmp/ipython-input-95312533.py:13: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_trader = pd.read_csv(TRADER_DATA_PATH)


✅ Data Loading Successful. Starting preprocessing...
✅ Data Merged. Starting final analysis.

--- Final Sentiment Performance Report ---
                          Days Observed  Avg. Total Daily PnL (USD)  \
Sentiment_Classification                                              
Extreme Fear                         14                    25483.72   
Fear                                 85                    20903.47   
Extreme Greed                       111                    13716.12   
Neutral                              66                    10571.52   
Greed                               184                     7601.84   

                          Daily Win Rate (%)  Avg. Daily Trades  \
Sentiment_Classification                                          
Extreme Fear                           64.29             124.71   
Fear                                   71.76             142.52   
Extreme Greed                          84.68              84.28   
Neutral                       

In [5]:
# Save to CSV for a professional deliverable
final_report.to_csv("sentiment_performance_report.csv")